In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：自定义培训文本二元分类模型用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署自定义文本二元分类模型进行批量预测。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB 电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)。您在本教程中将使用的数据集版本存储在公共云存储桶中。训练完成的模型预测评论的情感是积极还是消极。

### 目标

在本教程中，您将使用Vertex客户端库在Google预构建的Docker容器中从Python脚本创建一个带有训练流水线的自定义模型，然后对上传的模型进行批量预测。您也可以使用`gcloud`命令行工具或者在Google Cloud Console上在线创建自定义模型。

执行的步骤包括：

- 为训练模型创建一个Vertex自定义作业。
- 训练TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 进行批量预测。

费用

本教程使用 Google 云（GCP）的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您的预期使用量生成费用估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 GA 版本 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

安装完Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可以的话，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额以用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已在 Google Cloud Notebook 中安装。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本的其余操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您不能使用多区域存储桶来训练Vertex。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参见[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能会使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将其附加到在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，按提示进行身份验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，选择**存储对象管理员**。

点击创建。下载包含您的密钥的JSON文件到本地环境。

在下面的单元格中将您的服务帐户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您使用 Vertex 客户端库提交自定义训练任务时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 将从该包中运行代码。在本教程中，Vertex 还会将您的工作生成的训练模型保存在同一个存储桶中。然后，您可以根据这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目（包括您组织之外的项目）中是全局唯一的。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在的情况下才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查内容验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点 API 服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

设置用于训练和预测的硬件加速器（例如，GPU）。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU并分配给虚拟机（VM）实例的GPU数量的容器映像。例如，要使用一个具有4个Nvidia Tesla K80 GPU分配给每个VM的GPU容器映像，您将指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
 - aip.AcceleratorType.NVIDIA_TESLA_K80
 - aip.AcceleratorType.NVIDIA_TESLA_P100
 - aip.AcceleratorType.NVIDIA_TESLA_P4
 - aip.AcceleratorType.NVIDIA_TESLA_T4
 - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

*注意*：TF 2.3之前的版本由于在serving函数中生成的静态图操作而无法加载本教程中的自定义模型。这是一个已知问题，并在TF 2.3中修复--导致这个问题的原因。如果在您自己的自定义模型上遇到这个问题，请使用带有GPU支持的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的自定义模型和训练IMDB电影评论。

## 设置客户端

Vertex 客户端库采用客户/服务器模型。在您的一侧（Python 脚本）上，您将创建一个客户端，用于向 Vertex 服务器发送请求并接收回复。

在本教程中，您将为工作流中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于 `Model` 资源的模型服务。
- 用于部署的端点服务。
- 用于批处理任务和自定义训练的作业服务。
- 用于提供预测的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方式可使用容器镜像来训练自定义模型：

- **使用谷歌云预构建的容器**。如果使用预构建的容器，还需要指定要安装到容器镜像中的Python包。这个Python包包含用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用自己的容器，容器需要包含用于训练自定义模型的代码。

准备您的自定义工作规范

现在您的客户已经准备好了，您的第一步是为您的自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型及数量的规范（单一或分布式）
- `python_package_spec`：需要与预构建容器安装的Python软件包的规范。

### 准备您的机器规格

现在定义您定制训练任务的机器规格。这将告诉 Vertex 需要为训练提供什么类型的机器实例。
  - `machine_type`：要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
  - `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用 GPU；否则将使用 CPU。
  - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义您定制训练作业的磁盘规格。这会告诉 Vertex 在训练时为每台机器实例提供什么类型和大小的磁盘。

- `boot_disk_type`：可选择 SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小，单位为GB。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

定义工作池规范

接下来，您需要为自定义训练作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要配置该机器类型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在虚拟机实例上的 Python 训练包，以及要调用的 Python 模块，以及 Python 模块的命令行参数。

现在让我们深入了解 Python 包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的 Docker 镜像。

- `package_uris`：这是要安装在配置实例上的 Python 训练包的位置（URI）列表。这些位置需要在 Cloud Storage 存储桶中。这可以是单个 Python 文件或整个包的压缩文件（zip）。
    在后一种情况下，作业服务将解压缩（解压归档）内容到 Docker 镜像中。

- `python_module`：要调用以运行自定义训练作业的 Python 模块（脚本）。在本例中，您将调用 `trainer.task.py` -- 请注意不需要附加`.py`后缀。

- `args`：要传递给相应 Python 模块的命令行参数。在本例中，您将设置：
   - `"--model-dir=" + MODEL_DIR`：用于存储模型工件的 Cloud Storage 位置。有两种方法告诉训练脚本要保存模型工件的位置：
       - 直接：您将 Cloud Storage 位置作为命令行参数传递给您的训练脚本（设置变量 `DIRECT = True`），
       - 间接：服务将 Cloud Storage 位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您告知服务作业规范中的模型工件位置。
   - `"--epochs=" + EPOCHS`：训练的时期数。
   - `"--steps=" + STEPS`：每个时期的步骤（批处理）数量。
   - `"--distribute=" + TRAIN_STRATEGY"`：用于单设备或分布式训练的训练分发策略。
       - `"single"`：单设备。
       - `"mirror"`：单个计算实例上的所有 GPU 设备。
       - `"multi"`：所有计算实例上的所有 GPU 设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_imdb.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装完整的自定义工作规范描述：

- `display_name`：您为此自定义工作指定的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务云存储保存模型工件的位置（当变量`DIRECT = False`时）。然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将采用以下形式：
          
                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训套件

#### 套件布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 套件。解压缩后，该套件包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将该套件安装到 Docker 镜像的操作环境中的说明。

文件 `trainer/task.py` 是用于执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中提到它时，我们将目录斜杠替换为点(`trainer.task`)并且去掉文件后缀(`.py`)。

#### 套件组装

在以下单元格中，您将组装培训套件。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: IMDB Movie Reviews text binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py的内容

在下一个单元格中，您将编写训练脚本task.py的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TF Datasets（tfds）加载IMDB电影评论数据集。
- 使用TF.Keras模型API构建一个简单的RNN模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分发策略。
- 使用由`args.epochs`指定的时期训练模型（`fit()`）。
- 将经过训练的模型保存（`save(args.model_dir)`）到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for IMDB

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_rnn_model(encoder)

# Train the model
model.fit(train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

### 将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，参数如下：

- `parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将显示在`response`对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：为此自定义训练作业分配的Vertex完全合格标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它带有以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户服务的 `get_custom_job` 方法，并带有以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

如果你记得，在调用 `create_custom_job` 方法时，你在 `response.name` 字段中获得了自定义作业的 Vertex 完全限定标识符，并将其保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要20分钟甚至更长的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型实际花费的时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将其保存在您本地的云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载已保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 桶中。现在从 Cloud Storage 桶中加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来评估模型的表现如何。

### 加载评估数据

您将使用`tfds.datasets`中的`load()`方法加载IMDB电影评论测试（留存）数据。这将返回一个包含两个元素的元组。第一个元素是数据集，第二个是数据集的信息，其中包含预定义的词汇编码器。编码器将单词转换为数字嵌入，这些数字嵌入是预训练的，并在自定义训练脚本中使用。

当训练模型时，您需要为文本设置一个固定的输入长度。为了正向馈送批处理，相应的`tf.dataset`的`padded_batch()`属性被设置为将每个输入序列填充为相同形状以便形成一个批次。

对于测试数据，您还需要相应地设置`padded_batch()`属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 进行模型评估

现在评估定制作业中模型的表现如何。

In [ ]:
model.evaluate(test_dataset)

## 上传用于提供服务的模型

接下来，您将从自定义作业上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个提供服务的函数，将数据转换为您的模型所期望的格式。如果您将编码数据发送给Vertex，您的提供服务函数将确保在将数据作为输入传递给您的模型之前，在模型服务器上对数据进行解码。

### 提供服务的功能如何工作

当您向在线预测服务器发送请求时，该请求会被一个HTTP服务器接收。HTTP服务器从HTTP请求内容体中提取预测请求。提取出的预测请求被转发到提供服务的函数。对于Google预构建的预测容器，请求内容会作为`tf.string`传递给提供服务的函数。

提供服务的函数由两部分组成：

- `预处理函数`：
  - 将输入(`tf.string`)转换为基础模型(动态图)的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，归一化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题、创建JSON对象等。

预处理和后处理函数都会转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换的/打包的输出返回给HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建用于TF.Keras模型的提供服务函数时需要考虑的一点是，它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您在编译提供服务函数时会出错，指示您正在使用不受支持的EagerTensor。

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获得您模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 -- 等到您后续进行预测请求时会使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传你的模型，保存在SavedModel格式中，到`Model`服务，该服务将为你的模型实例化一个Vertex `Model`资源实例。一旦你完成了这个步骤，你就可以像其他Vertex `Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：用于`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：我们SavedModel工件的Cloud Storage路径。在本教程中，这是`trainer/task.py`保存模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：用于`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于你的模型是不需要Vertex `Dataset`资源构建的，你可以将其留空(`''`)。
- `artificat_uri`：模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将用于提供预测。使用你之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。调用`response.result()`，这是一个同步调用，当Vertex Model资源就绪时将返回。

辅助函数返回对应的Vertex Model实例`upload_model_response.model`的Vertex完全限定标识符。你将保存这个标识符到变量`model_to_deploy_id`中以备后续步骤使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model("imdb-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

获取“Model”资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数`get_model`，并提供以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

这个辅助函数调用 Vertex `Model` 客户端服务的方法`get_model`，并提供以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

批量预测模型部署

现在部署您为批量预测创建的已训练的Vertex“Model”资源。这与为按需预测部署“Model”资源不同。

对于在线预测，您需要：

1. 创建一个“Endpoint”资源来部署“Model”资源。

2. 将“Model”资源部署到“Endpoint”资源。

3. 向“Endpoint”资源发出在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求提供资源。

3. 返回批量预测请求的结果给调用者。

4. 作业服务将取消为批量预测请求提供资源。

## 发送批量预测请求

现在向您部署的模型发送批量预测请求。

### 准备请求内容

由于数据集是一个`tf.dataset`，它作为生成器，我们必须将其用作迭代器来访问测试数据中的数据项。我们按照以下步骤从测试数据中获取单个数据项：

- 使用`take(1)`方法将每次迭代要抽取的批次数量属性设置为一。
- 通过测试数据进行一次迭代 -- 即在for循环内部进行中断。
- 在单次迭代中，我们保存以元组形式表示的数据项。
- 数据项将是元组的第一个元素，然后您将其从张量转换为numpy数组 -- `data[0].numpy()`。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]

test_dataset.take(1)
for data in test_dataset:
    print(data)
    break

test_item = data[0].numpy()

### 制作批量输入文件

现在制作一个批量输入文件，你将把它存储在本地的云存储桶中。预测请求中的每个实例都是以下形式的字典条目：

                        {serving_input: content}

- `input_name`：底层模型输入层的名称。
- `content`：编码为嵌入的文本数据项。

In [ ]:
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {serving_input: test_item.tolist()}
    f.write(json.dumps(data) + "\n")

### 计算实例的扩展

处理批量预测请求时，您可以选择以下几种计算实例扩展方式：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，将会配置固定数量的计算实例，并且批量预测请求将均匀分布在这些实例上。

- 自动扩展：批量预测请求在可伸缩数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时需要配置和取消配置的实例数，并根据负载情况将计算实例的最大（`MAX_NODES`）数量设置为需要配置的数量。

计算实例的最小数量对应于字段`min_replica_count`，计算实例的最大数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 创建批量预测请求

现在您的两个测试项目的批次已经准备好了，让我们进行批量请求。使用以下参数调用这个帮助函数 `create_batch_prediction_job`：

- `display_name`: 预测作业的人类可读名称。
- `model_name`: Vertex 中用于 `Model` 资源的完全合格标识符。
- `gcs_source_uri`: 输入文件的 Cloud Storage 路径 -- 您之前创建的路径。
- `gcs_destination_output_uri_prefix`: 服务将把预测写入的 Cloud Storage 路径。
- `parameters`: 用于为服务预测结果添加额外筛选参数。

帮助函数调用作业客户服务的 `create_batch_prediction_job` 方法，使用以下参数：

- `parent`: Dataset、Model 和 Pipeline 资源的 Vertex 位置根路径。
- `batch_prediction_job`: 批量预测作业的规范。

现在让我们进入 `batch_prediction_job` 的规范：

- `display_name`: 预测批次作业的人类可读名称。
- `model`: Vertex 中用于 `Model` 资源的完全合格标识符。
- `dedicated_resources`: 为批量预测作业提供的计算资源。
  - `machine_spec`: 要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则仅分配 CPU。
  - `starting_replica_count`: 最初要提供的计算实例数，您之前设置的变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的最大计算实例数，您之前设置的变量 `MAX_NODES`。
- `model_parameters`: 用于为服务预测结果添加额外筛选参数。自定义模型不支持额外参数。
- `input_config`: 要预测的实例的输入源和格式类型。
 - `instances_format`: 批量预测请求文件的格式：`csv` 或 `jsonl`。
 - `gcs_source`: 您的批量预测请求的一个或多个 Cloud Storage 路径列表。
- `output_config`: 预测结果的输出目的地和格式。
 - `prediction_format`: 批量预测响应文件的格式：`csv` 或 `jsonl`。
 - `gcs_destination`: 预测结果的输出目的地。

此调用是一个异步操作。您将从响应对象中打印一些选择字段，包括：

- `name`: 分配给批量预测作业的 Vertex 完全合格标识符。
- `display_name`: 预测批次作业的人类可读名称。
- `model`: Model 资源的 Vertex 完全合格标识符。
- `generate_explanations`: 是否提供了与预测相关的 True/False 解释（可解释性）。
- `state`: 预测作业的状态（待处理、运行中等）。

由于这个调用需要一段时间来执行，您可能会得到 `JobState.JOB_STATE_PENDING` 作为 `state`。

In [ ]:
BATCH_MODEL = "imdb_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用以下辅助函数`get_batch_prediction_job`，并传递以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_batch_prediction_job`方法，并传递以下参数：

- `name`：批量预测作业的 Vertex 完全限定标识符。在本教程中，您将传递给它您的批量预测作业的 Vertex 完全限定标识符--`batch_job_id`

辅助函数将返回存储预测结果的 Cloud 存储路径-- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

获得预测

当批量预测完成处理时，作业状态将会是 `JOB_STATE_SUCCEEDED`。

最后，您可以查看保存在您设置为输出的 Cloud Storage 路径中的预测。这些预测将以 JSONL 格式存储，这是您创建批量预测作业时指定的，在以 `prediction` 开头的子文件夹下，文件夹中将会有一个名为 `prediction.results-xxxxx-of-xxxxx` 的文件。

现在显示（cat）其内容。您将会看到多个 JSON 对象，每个对象代表一个预测。

响应中包含每个实例的 JSON 对象，格式如下：

- `embedding_input`：预测的输入。
- `predictions` -- 预测的二项情感，介于 0（负面）和 1（正面）之间。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction.results*

        print("Results:")
        ! gsutil cat $folder/prediction.results*

        print("Errors:")
        ! gsutil cat $folder/prediction.errors*
        break
    time.sleep(60)

清理工作

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的以下各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME